# Testing Our Custom PCA Implementation

## Introduction

Now that we've built PCA from scratch, let's test it thoroughly to make sure it works correctly!

### What You'll Learn
1. How to use our custom PCA class
2. Verify implementation with manual calculations
3. Test all methods: fit, transform, inverse_transform
4. Visualize results
5. Test edge cases

### Testing Strategy
- Use simple 2D data for easy verification
- Compare with manual calculations
- Test different n_components settings
- Visualize transformations

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import our custom PCA implementation
from pca_implementation import PCA, plot_explained_variance, biplot

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.set_printoptions(precision=4, suppress=True)

print("✓ Libraries imported successfully!")
print(f"✓ Custom PCA module loaded from: {Path('pca_implementation.py').absolute()}")

## Test 1: Basic Functionality with 2D Data

Let's start with the same simple 2D dataset we used in the previous notebook.

In [ ]:
# Create test data
np.random.seed(42)
X = np.array([
    [2.5, 2.4],
    [0.5, 0.7],
    [2.2, 2.9],
    [1.9, 2.2],
    [3.1, 3.0],
    [2.3, 2.7],
    [2.0, 1.6],
    [1.0, 1.1]
])

print("Original Data:")
print(X)
print(f"\nShape: {X.shape}")

In [ ]:
# Initialize and fit PCA
pca = PCA(n_components=2)
pca.fit(X)

print("PCA Model:")
print(pca)
print("\nComponents (Principal axes):")
print(pca.components_)
print("\nMean:")
print(pca.mean_)

## Test 2: Compare with Manual Calculation

Let's verify our implementation matches manual PCA calculation.

In [ ]:
# Manual PCA calculation
print("Manual PCA Calculation:")
print("=" * 60)

# Step 1: Center data
mean_manual = X.mean(axis=0)
X_centered_manual = X - mean_manual
print(f"Mean: {mean_manual}")

# Step 2: Covariance matrix
cov_manual = np.cov(X_centered_manual.T)
print(f"\nCovariance matrix:\n{cov_manual}")

# Step 3: Eigenvalues and eigenvectors
eigenvalues_manual, eigenvectors_manual = np.linalg.eig(cov_manual)
idx = eigenvalues_manual.argsort()[::-1]
eigenvalues_manual = eigenvalues_manual[idx]
eigenvectors_manual = eigenvectors_manual[:, idx]

print(f"\nEigenvalues: {eigenvalues_manual}")
print(f"Eigenvectors:\n{eigenvectors_manual}")

# Step 4: Compare with our implementation
print("\n" + "=" * 60)
print("Comparison: Manual vs Our Implementation")
print("=" * 60)

print("\nMean match:", np.allclose(mean_manual, pca.mean_))
print("Explained variance match:", np.allclose(eigenvalues_manual, pca.explained_variance_))
print("Components match:", np.allclose(eigenvectors_manual.T, pca.components_))

if np.allclose(eigenvalues_manual, pca.explained_variance_):
    print("\n✓ SUCCESS: Our implementation matches manual calculation!")
else:
    print("\n✗ ERROR: Results don't match!")

## Test 3: Transform and Inverse Transform

In [ ]:
# Test transform
X_transformed = pca.transform(X)
print("Transformed Data (PCA space):")
print(X_transformed)
print(f"Shape: {X_transformed.shape}")

# Manual transformation
X_transformed_manual = X_centered_manual.dot(eigenvectors_manual)
print("\nManual Transformation:")
print(X_transformed_manual)

print("\nTransform match:", np.allclose(X_transformed, X_transformed_manual))

# Test inverse transform
X_reconstructed = pca.inverse_transform(X_transformed)
print("\nReconstructed Data (back to original space):")
print(X_reconstructed)

# Calculate reconstruction error
reconstruction_error = np.mean((X - X_reconstructed) ** 2)
print(f"\nReconstruction MSE: {reconstruction_error:.10f}")
print("Perfect reconstruction:", reconstruction_error < 1e-10)

if reconstruction_error < 1e-10:
    print("\n✓ SUCCESS: Perfect reconstruction (all components kept)!")

## Test 4: Dimensionality Reduction

Now let's test keeping only 1 component.

In [ ]:
# Reduce to 1D
pca_1d = PCA(n_components=1)
X_1d = pca_1d.fit_transform(X)

print("1D Reduction Results:")
print(f"Original shape: {X.shape}")
print(f"Reduced shape: {X_1d.shape}")
print(f"\nVariance explained: {pca_1d.explained_variance_ratio_[0]:.2%}")
print(f"\n1D Data:\n{X_1d.ravel()}")

# Reconstruct from 1D
X_reconstructed_1d = pca_1d.inverse_transform(X_1d)
reconstruction_error_1d = np.mean((X - X_reconstructed_1d) ** 2)

print(f"\nReconstruction MSE (1 component): {reconstruction_error_1d:.6f}")
print(f"Information loss: {(1 - pca_1d.explained_variance_ratio_[0]):.2%}")

In [ ]:
# Visualize 1D reduction and reconstruction
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot original vs reconstructed
ax1.scatter(X[:, 0], X[:, 1], s=150, alpha=0.7, color='blue',
           edgecolors='k', linewidths=2, label='Original', zorder=3)
ax1.scatter(X_reconstructed_1d[:, 0], X_reconstructed_1d[:, 1], s=150,
           alpha=0.7, color='red', marker='s', edgecolors='k', linewidths=2,
           label='Reconstructed (1 PC)', zorder=3)

# Draw reconstruction error lines
for i in range(len(X)):
    ax1.plot([X[i, 0], X_reconstructed_1d[i, 0]],
            [X[i, 1], X_reconstructed_1d[i, 1]],
            'k--', alpha=0.3, linewidth=1)

ax1.set_xlabel('Feature 1', fontsize=12)
ax1.set_ylabel('Feature 2', fontsize=12)
ax1.set_title('Reconstruction from 1 Component', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Plot 1D representation
ax2.scatter(X_1d, np.zeros_like(X_1d), s=150, alpha=0.7,
           c=range(len(X_1d)), cmap='viridis', edgecolors='k', linewidths=2)
for i, val in enumerate(X_1d.ravel(), 1):
    ax2.annotate(f'S{i}', (val, 0), xytext=(0, 10),
                textcoords='offset points', ha='center')
ax2.axhline(0, color='gray', linewidth=2)
ax2.set_xlabel('PC1 Value', fontsize=12)
ax2.set_yticks([])
ax2.set_title(f'1D Representation ({pca_1d.explained_variance_ratio_[0]:.1%} variance)',
             fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## Test 5: Variance Threshold

Test automatic component selection based on variance threshold.

In [ ]:
# Test with variance threshold
print("Testing variance threshold selection:")
print("=" * 60)

for threshold in [0.90, 0.95, 0.99]:
    pca_threshold = PCA(n_components=threshold)
    pca_threshold.fit(X)
    
    print(f"\nThreshold: {threshold:.0%}")
    print(f"  Components selected: {pca_threshold.n_components_}")
    print(f"  Actual variance: {pca_threshold.explained_variance_ratio_.sum():.2%}")

## Test 6: Utility Functions

Test the visualization utility functions.

In [ ]:
# Test plot_explained_variance
plot_explained_variance(pca)

In [ ]:
# Test biplot
feature_names = ['Nitrogen (ppm)', 'Phosphorus (ppm)']
biplot(pca, X, feature_names=feature_names)

## Test 7: Higher Dimensional Data

Test with more realistic multi-dimensional data.

In [ ]:
# Generate 5D correlated data
np.random.seed(42)
n_samples = 100
n_features = 5

# Create correlated features
base = np.random.randn(n_samples, 2)
X_5d = np.column_stack([
    base[:, 0] + np.random.randn(n_samples) * 0.1,
    base[:, 0] * 0.8 + np.random.randn(n_samples) * 0.2,
    base[:, 1] + np.random.randn(n_samples) * 0.1,
    base[:, 1] * 0.7 + np.random.randn(n_samples) * 0.3,
    np.random.randn(n_samples) * 0.5  # Independent feature
])

print(f"5D Data shape: {X_5d.shape}")
print(f"\nFirst 5 samples:\n{X_5d[:5]}")

In [ ]:
# Fit PCA on 5D data
pca_5d = PCA(n_components=5)
X_5d_transformed = pca_5d.fit_transform(X_5d)

print("5D PCA Results:")
print("=" * 60)
print(f"\nExplained variance by component:")
for i, var in enumerate(pca_5d.explained_variance_ratio_, 1):
    print(f"  PC{i}: {var:.2%}")

cumsum = np.cumsum(pca_5d.explained_variance_ratio_)
print(f"\nCumulative variance:")
for i, var in enumerate(cumsum, 1):
    print(f"  First {i} PCs: {var:.2%}")

# Find components needed for 95% variance
n_95 = np.argmax(cumsum >= 0.95) + 1
print(f"\n✓ {n_95} components needed for 95% variance")

In [ ]:
# Plot explained variance for 5D data
plot_explained_variance(pca_5d)

In [ ]:
# Visualize first 2 PCs
plt.figure(figsize=(10, 8))
plt.scatter(X_5d_transformed[:, 0], X_5d_transformed[:, 1],
           alpha=0.6, s=50, edgecolors='k', linewidths=0.5,
           c=range(n_samples), cmap='viridis')
plt.xlabel(f'PC1 ({pca_5d.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca_5d.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title('5D Data Projected onto First 2 Principal Components', fontsize=14, fontweight='bold')
plt.colorbar(label='Sample index')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Test 8: Additional Methods

In [ ]:
# Test get_covariance
cov_estimated = pca.get_covariance()
cov_actual = np.cov(X.T)

print("Covariance Matrix Estimation:")
print("\nActual covariance:")
print(cov_actual)
print("\nEstimated covariance:")
print(cov_estimated)
print("\nClose match:", np.allclose(cov_actual, cov_estimated, rtol=0.1))

In [ ]:
# Test score method
score = pca.score(X)
print(f"Score (negative reconstruction error): {score:.6f}")
print("\nNote: Higher score is better (closer to 0 = better reconstruction)")

## Test 9: Edge Cases

In [ ]:
# Test with n_components = None
pca_all = PCA(n_components=None)
pca_all.fit(X)
print(f"n_components=None: Kept {pca_all.n_components_} components (all)")

# Test with n_components > n_features
pca_large = PCA(n_components=10)
pca_large.fit(X)
print(f"\nn_components=10 (> n_features): Kept {pca_large.n_components_} components (max possible)")

# Test error handling
try:
    pca_uninit = PCA()
    pca_uninit.transform(X)  # Should raise error
except ValueError as e:
    print(f"\n✓ Proper error handling: {e}")

## Test Summary

In [ ]:
print("="*70)
print(" " * 20 + "TEST SUMMARY")
print("="*70)

tests = [
    "✓ Basic functionality (fit, transform)",
    "✓ Manual calculation verification",
    "✓ Transform and inverse transform",
    "✓ Dimensionality reduction (2D → 1D)",
    "✓ Variance threshold selection",
    "✓ Utility functions (plotting)",
    "✓ Higher dimensional data (5D)",
    "✓ Additional methods (covariance, score)",
    "✓ Edge cases and error handling"
]

for test in tests:
    print(test)

print("="*70)
print("\n🎉 ALL TESTS PASSED! Our PCA implementation is working correctly!")
print("\nReady to compare with sklearn in the next notebook.")
print("="*70)

## Key Takeaways

### What We Verified

1. **Correctness**: Our implementation matches manual calculations
2. **Completeness**: All methods work as expected
3. **Robustness**: Handles edge cases properly
4. **Flexibility**: Works with different dimensions and parameters

### Understanding Gained

- PCA is fundamentally about eigendecomposition
- The choice of n_components involves trade-offs
- Reconstruction error quantifies information loss
- Visualization helps understand what PCA does

### Next Steps

Now we'll:
1. Learn to use sklearn's optimized PCA
2. Compare our implementation with sklearn
3. Apply PCA to real agricultural data

---

**Excellent work!** You've thoroughly tested a PCA implementation from scratch.

Continue to: `../3_with_sklearn/sklearn_pca_basics.ipynb`